In [1]:
import requests;
import pandas as pd;
from io import BytesIO;

## Dataset Discovery

In [2]:
response = requests.get("https://api.beta.ons.gov.uk/v1/datasets");

datasets = response.json().get("items");

In [3]:
print(datasets[0].keys());

dict_keys(['contacts', 'description', 'keywords', 'id', 'last_updated', 'links', 'methodologies', 'national_statistic', 'next_release', 'qmi', 'related_datasets', 'release_frequency', 'state', 'title', 'unit_of_measure'])


In [7]:
for dataset in datasets:
    datasetId = dataset.get("id");
    title = dataset.get("title");
    lastUpdated = dataset.get("last_updated");
    nextRelease = dataset.get("next_release");
    releaseFrequency = dataset.get("release_frequency");

    print(f'{datasetId:s}, {title:s}, {lastUpdated:s}, {nextRelease:s}, {releaseFrequency:s}');

wellbeing-quarterly, Quarterly personal well-being estimates, 2023-12-13T09:40:24.204Z, TBC, Quarterly
wellbeing-local-authority, Personal well-being estimates by local authority, 2023-12-13T09:40:21.928Z, TBC, Annual
weekly-deaths-region, Deaths registered weekly in England and Wales by region, 2025-08-13T09:32:31.24Z, 20 August 2025, Weekly
weekly-deaths-local-authority, Death registrations and occurrences by local authority and place of death, 2024-01-09T10:10:57.755Z, 17 January 2024, Weekly
weekly-deaths-health-board, Death registrations and occurrences by health board and place of death, 2024-01-09T10:11:08.441Z, 17 January 2024, Weekly
weekly-deaths-age-sex, Deaths registered weekly in England and Wales by age and sex, 2025-08-13T09:32:27.636Z, 20 August 2025, Weekly
uk-spending-on-cards, UK spending on credit and debit cards, 2024-05-16T09:00:13.637Z, To be announced, Weekly
uk-business-by-enterprises-and-local-units, UK Business: Activity, Size and Location, 2022-11-03T09:50:0

In [7]:
trade = next(filter(lambda d: d.get("id") == "trade", datasets), None);
latestTradeDatasetLink = trade.get('links').get('latest_version').get('href');
tradeDatasetResponse = requests.get(latestTradeDatasetLink);

print(tradeDatasetResponse.json());

{'alerts': [], 'collection_id': 'cmdtrade-284a77f416fe8409d6f3b92a1d8a72669ee97749bacd7221825f7faf33b41cb4', 'dimensions': [{'label': 'time', 'links': {'code_list': {}, 'options': {}, 'version': {}}, 'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mmm-yy', 'id': 'mmm-yy', 'name': 'time'}, {'label': 'geography', 'links': {'code_list': {}, 'options': {}, 'version': {}}, 'href': 'https://api.beta.ons.gov.uk/v1/code-lists/uk-only', 'id': 'uk-only', 'name': 'geography'}, {'label': 'standardindustrialtradeclassification', 'links': {'code_list': {}, 'options': {}, 'version': {}}, 'href': 'https://api.beta.ons.gov.uk/v1/code-lists/sitc', 'id': 'sitc', 'name': 'standardindustrialtradeclassification'}, {'label': 'countriesandterritories', 'links': {'code_list': {}, 'options': {}, 'version': {}}, 'href': 'https://api.beta.ons.gov.uk/v1/code-lists/countries-and-territories', 'id': 'countries-and-territories', 'name': 'countriesandterritories'}, {'label': 'direction', 'links': {'code_list': {}, 

In [8]:
tradeDatasetLink = tradeDatasetResponse.json().get("downloads").get("csv").get("href")
tradeDataset = requests.get(tradeDatasetLink)

In [9]:
pd.read_csv(BytesIO(tradeDataset.content))

,v4_0,mmm-yy,Time,uk-only,Geography,sitc,StandardIndustrialTradeClassification,countries-and-territories,CountriesAndTerritories,trade-direction,Direction
0,3237.00,Jan-18,Jan-18,K02000001,United Kingdom,0,0: Food & live animals,W1,W1 - Whole world,IM,Imports
1,312.00,Jan-18,Jan-18,K02000001,United Kingdom,07,"07: Coffee, tea, cocoa etc",W1,W1 - Whole world,IM,Imports
2,175.00,Jan-18,Jan-18,K02000001,United Kingdom,08,08: Animal feeding stuffs,W1,W1 - Whole world,IM,Imports
3,865.00,Jan-18,Jan-18,K02000001,United Kingdom,2,2: Crude materials,W1,W1 - Whole world,IM,Imports
4,2.00,Jan-18,Jan-18,K02000001,United Kingdom,21,"21: Hides, skins & furskins",W1,W1 - Whole world,IM,Imports
...,...,...,...,...,...,...,...,...,...,...,...
5291455,0.30,Jun-25,Jun-25,K02000001,United Kingdom,8,8: Miscellaneous manufactures,ZW,ZW - Zimbabwe,EX,Exports
5291456,0.00,Jun-25,Jun-25,K02000001,United Kingdom,83,"83: Travel goods, handbags etc",ZW,ZW - Zimbabwe,EX,Exports
5291457,0.41,Jun-25,Jun-25,K02000001,United Kingdom,7MI,7MI: Mechanical machinery (intermediate),ZW,ZW - Zimbabwe,EX,Exports
5291458,0.00,Jun-25,Jun-25,K02000001,United Kingdom,85,85: Footwear,ZW,ZW - Zimbabwe,EX,Exports


# Dataset filtering
Need to first create a filter then query it. The 

In [24]:
datasetFilterUrl = f'https://api.beta.ons.gov.uk/v1/filters';
filterPayload = {
        "dataset": {
            "id": "trade",
            "edition": "time-series", 
            "version": "59"
        },
        "dimensions": [
            {
                "name": "time",
                "options": ["2023-01", "2023-02", "2023-03"]
            }
        ]
        
    };

response = requests.post(datasetFilterUrl, json=filterPayload)
print(response)

<Response [500]>


<Response [404]>
